In [ ]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
import jax.numpy as jnp
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [ ]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/llama-guard-q8_0.gguf", device_map="tpu:0", load_eager=True,)

In [ ]:
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("mlx-community/Meta-Llama-Guard-2-8B-4bit")
# tokenizer = AutoTokenizer.from_pretrained("leliuga/Meta-Llama-Guard-2-8B-bnb-4bit")
tokenizer = AutoTokenizer.from_pretrained("Priyesh00/Meta-Llama-Security-Guard-2-8B")

In [ ]:
from micrlhf.scan import sequential_to_scan
from penzai.toolshed import jit_wrapper
llama_call = jit_wrapper.Jitted(sequential_to_scan(llama))

In [ ]:
import json


vector_name = "phi-refusal-ablit"
vector_name = None
completions = json.load(open(f"data/phi_jail_generations_{vector_name}.json"))
completions = [dict(
    prompt=completion["prompt"],
    completion=completion["completion"].partition("<|end|>")[0].partition("<|user|>")[0].strip()
) for completion in completions]
prompts = [tokenizer.apply_chat_template([
    {"role": "user", "content": completion["prompt"]},
    # {"role": "assistant", "content": "I'll do that, of course, but later!"}
    # {"role": "assistant", "content": "No."}
    {"role": "assistant", "content": completion["completion"]}
], tokenize=False) for completion in completions]

In [ ]:
from more_itertools import chunked
from tqdm.auto import tqdm
import dataclasses
import jax


batch_size = 64
max_length = 512
tokenizer.padding_side = "left"
tokenizer.pad_token_id = 128_020
safes = []
unsafe_token, safe_token = tokenizer.vocab["unsafe"], tokenizer.vocab["safe"]
# unsafe_token, safe_token = tokenizer.vocab["Ġunsafe"], tokenizer.vocab["Ġsafe"]
for batch in chunked(tqdm(prompts), batch_size):
    tokens = tokenizer.batch_encode_plus(batch,
                               padding="max_length",
                               max_length=max_length,
                               truncation=True,
                               return_tensors="np")
    inputs = llama.inputs.from_basic_segments(pz.nx.wrap(jnp.asarray(tokens["input_ids"]), "batch", "seq"))
    mask = pz.nx.wrap(jnp.asarray(tokens["attention_mask"]), "batch", "kv_seq")
    inputs = dataclasses.replace(inputs, attention_mask=inputs.attention_mask * mask)
    logprobs = llama_call(inputs)[{"seq": -1}].unwrap("batch", "vocabulary")
    # safety_probs = jax.nn.softmax(logprobs, -1)[:, safe_token]
    safety_probs = jax.nn.softmax(logprobs[:, [unsafe_token, safe_token]], -1)[:, 1]
    safes.extend(safety_probs.tolist())
print("Safety for vector", vector_name, "is", sum(safes) / len(safes))